In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import glob

import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [ ]:
import tensorflow as tf

In [ ]:
file_names = glob.glob('lovecraft_corpus/*.txt')

In [ ]:
len(file_names)

In [ ]:
def get_sentences(file_name):
    with open(file_name, 'r') as f:
        return f.read().split('.')

In [ ]:
MIN_LENGTH = 90
sentences = []
for file_name in file_names:
    sentences+=get_sentences(file_name)

sentences = [sentence.replace('\n','') for sentence in sentences]
sentences = [sentence.replace('\t','') for sentence in sentences]
sentences = [sentence for sentence in sentences if len(sentence)>MIN_LENGTH]

In [ ]:
lengths = [len(sentence) for sentence in sentences]

In [ ]:
lengths = pd.Series(lengths)

In [ ]:
lengths.describe()
# 14228 our of 18k

In [ ]:
sentences

# Load Whole Corpus

In [ ]:
corpus = ""
for file_name in file_names:
    with open(file_name, 'r') as f:
            corpus+=f.read()
corpus = corpus.replace('\n','')
corpus = corpus.replace('\t','')
corpus = corpus.replace('æ','ae')
corpus = corpus.replace('ë','e')
corpus = corpus.replace('ö','o')
corpus = corpus.replace('ä','a')

len(corpus)

In [ ]:
len(corpus)

In [ ]:
chars = list(set(corpus))
VOCAB_SIZE = len(chars)
char_to_ix = {char:ix for ix, char in enumerate(chars)}
SEQ_LENGTH = 50
slices = len(corpus)//SEQ_LENGTH

In [ ]:
slices

In [ ]:
import sys

sys.getsizeof('a'*SEQ_LENGTH)*slices

In [ ]:
sorted(chars)

In [ ]:
X = np.zeros((slices, SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((slices, SEQ_LENGTH, VOCAB_SIZE))

for i in range(0, slices):
    if i% 500==0:
        print(i)
    X_sequence = corpus[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
    y_sequence = corpus[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

In [ ]:
sys.getsizeof(X)

# Two LSTM layers

In [ ]:
UNITS = 125 #75
TOTAL_OUTPUT = SEQ_LENGTH*VOCAB_SIZE
inputs = tf.keras.Input(shape=(SEQ_LENGTH,VOCAB_SIZE), name='sentences')

x = tf.keras.layers.LSTM(units = UNITS, name='LSTM_layer_1', return_sequences=True)(inputs)
x = tf.keras.layers.LSTM(units = UNITS, name='LSTM_layer_2', return_sequences=True)(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(VOCAB_SIZE))(x)
outputs = tf.keras.layers.Dense(VOCAB_SIZE, activation='softmax', name='predicted_sentence')(x)
outputs = tf.keras.layers.Reshape((SEQ_LENGTH,VOCAB_SIZE))(outputs)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

In [ ]:
customAdam = tf.keras.optimizers.Adam(lr=0.0005) 
model.compile(optimizer=customAdam, 
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["mean_squared_error","binary_crossentropy"])
es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience = 5)

# Train the model

In [ ]:
print('# Fit model on training data')

history = model.fit(X, 
                    y, #we pass it the same input data as desired output
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size=4096, 
                    epochs=500,
                    # using a sample as validation data. This is kinda wrong.
                    validation_split=0.1,
                    callbacks = [es]
                   )

In [ ]:
model.save('model_50_150_3.h5')

# Dropout + 2 LSTM Layers 

In [ ]:
X.shape

In [ ]:
chars_np = np.asarray(chars)

def output_idx(i):
    return np.argmax(model.predict(np.asarray([X[i]]))[0], 1)

def output_str(i):
    return ''.join(list(chars_np[output_idx(i)]))

In [ ]:
random_stuff = ''
for i in range(1000):
    random_stuff+= output_str(i)
with open('random_output_150_50_3.txt','w') as f:
    f.write(random_stuff)

In [ ]:
random_stuff

In [ ]:
from collections import Counter
output_char_counts = Counter(random_stuff)
input_char_counts = Counter(corpus)

output_length = len(random_stuff)
corpus_length = len(corpus)

for k,v in input_char_counts.items():
    print("{}: {}, {}".format(k,100.0*v/corpus_length,100.0*output_char_counts[k]/output_length))

In [ ]:
characters_in_corpus = []
input_share = []
output_share = []
for k,v in input_char_counts.items():
    characters_in_corpus.append(k)
    input_share.append(100.0*v/corpus_length)
    output_share.append(100.0*output_char_counts[k]/output_length)


In [ ]:
df = pd.DataFrame()
df['char'] = characters_in_corpus
df['input_share'] = input_share
df['output_share'] = output_share

In [ ]:
output_str(100)

In [ ]:
pd.Series([len(word) for word in random_stuff.split(' ')]).mean()

In [ ]:
pd.Series([len(word) for word in corpus.split(' ')]).mean()

In [ ]:
def output_idx_random():
    X_sequence_ix = np.random.randint(0, VOCAB_SIZE, size=SEQ_LENGTH)
    
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    
    out_idx = np.argmax(model.predict([[input_sequence]])[0], 1)
    return ''.join(list(chars_np[out_idx]))

In [ ]:
output_idx_random()

## LSTM with less characters

# predict only 10

In [ ]:
INPUT_LENGTH = 10
OUTPUT_LENGTH = 10
slices = len(corpus)//INPUT_LENGTH
X_smaller = np.zeros((slices, INPUT_LENGTH, VOCAB_SIZE))
y_smaller = np.zeros((slices, OUTPUT_LENGTH, VOCAB_SIZE))

for i in range(0, slices):
    if i% 500==0:
        print(i)
    X_sequence = corpus[i*INPUT_LENGTH:(i+1)*INPUT_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((INPUT_LENGTH, VOCAB_SIZE))
    for j in range(INPUT_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X_smaller[i] = input_sequence
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
    y_sequence = corpus[(i+1)*INPUT_LENGTH:(i+1)*INPUT_LENGTH+OUTPUT_LENGTH]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((OUTPUT_LENGTH, VOCAB_SIZE))
    for j in range(OUTPUT_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y_smaller[i] = target_sequence

In [ ]:
UNITS = 125 #75
TOTAL_OUTPUT = OUTPUT_LENGTH*VOCAB_SIZE
inputs_s = tf.keras.Input(shape=(INPUT_LENGTH,VOCAB_SIZE), name='sentences')

x_s = tf.keras.layers.LSTM(units = UNITS, name='LSTM_layer_1', return_sequences=True)(inputs_s)
x_s = tf.keras.layers.LSTM(units = UNITS*2, name='LSTM_layer_2', return_sequences=True)(x_s)
x_s = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(VOCAB_SIZE))(x_s)
outputs_s = tf.keras.layers.Dense(VOCAB_SIZE, activation='softmax', name='predicted_sentence')(x_s)


outputs_s = tf.keras.layers.Reshape((OUTPUT_LENGTH,VOCAB_SIZE))(outputs_s)
model_10 = tf.keras.models.Model(inputs=inputs_s, outputs=outputs_s)

In [ ]:
customAdam = tf.keras.optimizers.Adam(lr=0.001) 
model_10.compile(optimizer=customAdam, 
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["mean_squared_error","binary_crossentropy"])
es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience = 5)

In [ ]:
print('# Fit model on training data')

history = model_10.fit(X_smaller, 
                    y_smaller, #we pass it the same input data as desired output
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size=256, 
                    epochs=500,
                    # using a sample as validation data. This is kinda wrong.
                    validation_split=0.1,
                    callbacks = [es]
                   )

In [ ]:
X_smaller.shape

In [ ]:
y_smaller.shape

In [ ]:
model_10.summary()